In [1]:
import os
os.chdir(os.getcwd().rsplit(os.path.sep + 'code')[0] + os.path.sep + 'code')
from importlib.machinery import SourceFileLoader
setup = SourceFileLoader("setup", "./00_setup.py").load_module()

# XGBoost, Modified DGI Hierarchical - Additional Start Levels
Do an alternative hierarchical blending, NAICS + hierarcy above a certain level
Use all groups

Copy of 67 with lambda_k = 100

*This script takes about 5 minutes on my MacBook Air*

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import warnings
import matplotlib.pyplot as plt
import pickle

In [3]:
from pathlib import Path

In [4]:
import sklearn as sk
from sklearn.model_selection import RandomizedSearchCV
from sklearn.inspection import permutation_importance, partial_dependence
from scikitplot.metrics import plot_lift_curve

In [5]:
from sba_gnn.sba_gnn import sg_plot, sg_blender
from sba_gnn.sba_gnn.sg_blender import HierarchicalEncoder # Target encoder

## Import Processed Datasets

In [6]:
sba_loans = pd.read_parquet(Path(setup.temp_path).joinpath('01_DATA_transformed.parquet')) 

In [7]:
predictor_features = setup.predictor_features + ['mhier_NAICS_alt']

In [8]:
best_params_df = pd.read_csv(Path(setup.temp_path).joinpath('03_REPORT_fit_parameter_selected.csv'))
best_params = best_params_df.to_dict(orient='records')[0]
best_params

{'subsample': 0.8,
 'scale_pos_weight': 1.0,
 'reg_lambda': 0.01,
 'reg_alpha': 0.001,
 'min_child_weight': 50,
 'max_depth': 8,
 'learning_rate': 0.1,
 'gamma': 2}

In [9]:
xgb_params = dict(best_params, 
                   **{'objective':"binary:logistic", 'n_estimators':setup.xgb_n_estimators})

In [10]:
embed_df = pd.read_parquet(Path(setup.temp_path).joinpath('63_DATA_embeddings_tsne_naics.parquet'))

##### NAICS levels to select NAICS-like clusters

In [11]:
naics_grp_stats = pd.read_csv(Path(setup.temp_path).joinpath('02_REPORT_naics_grp_stats_all.csv'))
naics_grp_k = naics_grp_stats['count_grp'].drop_duplicates().sort_values().to_list()
naics_grp_k

[20.0, 106.0, 354.0, 834.0]

In [12]:
cluster_cols = sorted([c for c in embed_df if c.startswith('cluster_')],
                      reverse=True)
cluster_cols

['cluster_834',
 'cluster_354',
 'cluster_106',
 'cluster_020',
 'cluster_010',
 'cluster_003']

In [13]:
sba_loans = sba_loans.drop(columns = cluster_cols, errors='ignore') \
    .merge(embed_df[['NAICS_orig'] + cluster_cols] \
               .rename(columns={'NAICS_orig':'NAICS'}),
           on='NAICS', how='left')

## Function to get metrics

In [14]:
def get_metrics(data, enc_features = ['NAICS', 'NAICS_sector']):
    
    # Alternative encoding
    train_df = data[data['dset'] == 'train']
    hier_enc = HierarchicalEncoder(lambda_k = 100)
    hier_enc.fit(train_df[enc_features],train_df['target'])
    hier_col = hier_enc.transform(data[enc_features]) \
        .rename('mhier_NAICS_alt')
    
    # Append to data
    data = pd.concat([data.drop(columns='mhier_NAICS_alt', errors='ignore'), 
                       hier_col], axis=1)
    train_df = data[data['dset'] == 'train']
    test_df = data[data['dset'] == 'test']
    val_df = data[data['dset'] == 'val']
    
    # Model fit
    xgb_model = xgb.XGBClassifier(objective="binary:logistic",
                             n_estimators =  setup.xgb_n_estimators)
    xgb_model = xgb_model.set_params(**xgb_params)
    xgb_model.fit(train_df[predictor_features], train_df['target'],
              eval_set = [(train_df[predictor_features], train_df['target']),
                         (val_df[predictor_features], val_df['target'])])
    
    # Predict
    pred_df = pd.concat([data[['LoanNr_ChkDgt', 'dset', 'target', 'dset_naics_holdout']].reset_index(),
                     pd.DataFrame({'predict_prob':
                                   xgb_model.predict_proba(data[predictor_features])[:,1]})],
                    axis=1) \
    .set_index('index')
    
    # Decision threshold
    pred_train_df = pred_df[pred_df['dset'] == 'train']
    thresh_tune_data = sg_plot.get_f1_frame(pred_train_df['target'], 
                                        pred_train_df['predict_prob'])
    thresh_head = thresh_tune_data.sort_values('f1', ascending=False).head(2)
    best_thresh = thresh_head['thresh'].iloc[0]
    pred_df['predict_bin'] = np.where(pred_df['predict_prob'] >= best_thresh, 1, 0)
    
    # Metrics
    metrics_dset_df = pred_df.groupby('dset') \
        .apply(lambda x: sg_plot.dset_metrics(x.target, x.predict_bin, x.predict_prob)) \
        .reset_index()
    metrics_test_df = pred_df[pred_df['dset'] == 'test'] \
        .groupby(['dset', 'dset_naics_holdout']) \
        .apply(lambda x: sg_plot.dset_metrics(x.target, x.predict_bin, x.predict_prob)) \
        .reset_index()
    metrics_df = pd.concat([metrics_dset_df, metrics_test_df])
    
    return pred_df, metrics_df

## Loop to get predictions, metrics

In [15]:
# Hierarchical encode from different levels
naics_sets = [['NAICS'] + cluster_cols[i:] \
              for i in range(len(cluster_cols))]
naics_sets

[['NAICS',
  'cluster_834',
  'cluster_354',
  'cluster_106',
  'cluster_020',
  'cluster_010',
  'cluster_003'],
 ['NAICS',
  'cluster_354',
  'cluster_106',
  'cluster_020',
  'cluster_010',
  'cluster_003'],
 ['NAICS', 'cluster_106', 'cluster_020', 'cluster_010', 'cluster_003'],
 ['NAICS', 'cluster_020', 'cluster_010', 'cluster_003'],
 ['NAICS', 'cluster_010', 'cluster_003'],
 ['NAICS', 'cluster_003']]

In [16]:
%%capture
pred_df = pd.DataFrame()
metrics_df = pd.DataFrame()
for this_set in naics_sets:
    this_pred, this_met = get_metrics(sba_loans, this_set)
    this_pred['start_naics'] = this_set[1]
    pred_df = pd.concat([pred_df, this_pred])
    this_met['start_naics'] = this_set[1]
    metrics_df = pd.concat([metrics_df, this_met])

In [17]:
metrics_df[metrics_df['dset_naics_holdout'] == 1]

,dset,accuracy_score,f1_score,precision_score,recall_score,average_precision_score,roc_auc_score,dset_naics_holdout,start_naics
1,test,0.705067,0.400175,0.317382,0.541409,0.331153,0.718490,1.0,cluster_834
1,test,0.707234,0.399117,0.318255,0.535065,0.334981,0.723500,1.0,cluster_354
1,test,0.678503,0.421338,0.313063,0.644108,0.347546,0.730198,1.0,cluster_106
1,test,0.663469,0.429331,0.310275,0.696642,0.359413,0.738050,1.0,cluster_020
1,test,0.700209,0.424176,0.325804,0.607647,0.356346,0.736042,1.0,cluster_010
1,test,0.686113,0.421673,0.316955,0.629727,0.352802,0.732988,1.0,cluster_003


In [18]:
metrics_df[metrics_df['dset_naics_holdout'] == 0]

,dset,accuracy_score,f1_score,precision_score,recall_score,average_precision_score,roc_auc_score,dset_naics_holdout,start_naics
0,test,0.658328,0.449114,0.335983,0.677106,0.382840,0.728241,0.0,cluster_834
0,test,0.658049,0.448799,0.335707,0.676793,0.382590,0.727930,0.0,cluster_354
0,test,0.633364,0.450351,0.325571,0.730219,0.383011,0.728348,0.0,cluster_106
0,test,0.632283,0.450257,0.325102,0.732092,0.383184,0.728500,0.0,cluster_020
0,test,0.659558,0.449722,0.336857,0.676325,0.382765,0.728171,0.0,cluster_010
0,test,0.659355,0.448620,0.336267,0.673724,0.383069,0.728175,0.0,cluster_003


In [19]:
pred_df.to_parquet(Path(setup.temp_path).joinpath('X67_DATA_combined_predictions.parquet'))
metrics_df.to_csv(Path(setup.temp_path).joinpath('X67_REPORT_metrics.csv'), index=True)